Import Libs

In [1]:
import os
import geopandas as gpd
from osgeo import gdal, gdal_array
from osgeo import ogr
import rasterio
from rasterio import plot
from rasterio.mask import mask
from rasterio.features import geometry_mask
from rasterio.enums import Resampling
from rasterio.warp import reproject
from rasterio.warp import calculate_default_transform, reproject
from rasterio.transform import Affine
from rasterio.plot import show
import rasterio.transform as rt
from matplotlib import pyplot
import fiona
import shapely
import glob
import geopandas as gpd
import rioxarray
import xarray as xr
import pyproj
import numpy as np
import pandas as pd
import cv2

This line of code is for cliping the input dataset tif files accoridng to the target area and save it as thier unit_code

In [30]:
import geopandas as gpd
import pandas as pd

# Load the shapefile
shapefile_path = "D:\\PhD_Udel\\MICRA_2015\\DOwnscaling\\Africa\\Shapefile\\Shapefile.shp"
shapefile = gpd.read_file(shapefile_path)
# Load the Excel file
excel_path = "D:\\PhD_Udel\\MICRA_2015\\DOwnscaling\\Africa\\CCC\\New CCC\\New folder\\GAEZ\\New\\unit_code.xlsx"  # Update with your Excel file path
excel_data = pd.read_excel(excel_path)
# Merge the shapefile with the Excel data on the 'COUNTRY' column
merged_shapefile = shapefile.merge(excel_data[['COUNTRY', 'unit_code']], on='COUNTRY', how='left')

# Check the merged DataFrame columns
print("Merged shapefile columns:", merged_shapefile.columns)

# Directly update 'unit_code' in the shapefile
# This will replace the existing 'unit_code' with the values from the Excel file
# where the country names match
if 'unit_code' in merged_shapefile.columns:
    shapefile['unit_code'] = merged_shapefile['unit_code']
    print("unit_code column updated successfully.")
else:
    print("Error: 'unit_code' column not found in the merged shapefile.")

# Save the updated shapefile
updated_shapefile_path =  "D:\\PhD_Udel\\MICRA_2015\\DOwnscaling\\Africa\\Shapefile\\Final.shp"
shapefile.to_file(updated_shapefile_path)

print("Shapefile has been successfully updated and saved.")

Loop over each subfolder, read the file with .tif and .shp extension, clip the raster per each polygon using a unit_code and save by the creating a folder after each raster name

Loop over each subfolder to read the crop calander .nc4 files, convert it to tif and resample it with 5 arc-minutes resolution

In [5]:
root_folder = r"D:\\PhD_Udel\\MICRA_2015\\Input Data\\Crop_Calander\\NetCDF"
crs = pyproj.CRS.from_epsg(4326)
Resam_fold = r"D:\\PhD_Udel\\Crop_calendar\\Resample"
xres, yres = 0.00833, 0.00833
compression = "LZW"

for subdir, dirs, files in os.walk(root_folder):
    for file in files:
        if file.endswith('.nc4'):
            nc_file = xr.open_dataset(os.path.join(subdir, file))
            var_planting = nc_file.variables["planting_day"]
            var_maturity = nc_file.variables["maturity_day"]
            
            x = nc_file.coords['lon'].values
            y = nc_file.coords['lat'].values
            left, bottom = x.min(), y.min()
            right, top = x.max(), y.max()
            nx = int((right - left) / xres)
            ny = int((top - bottom) / yres)
            
            transform = rt.from_origin(left, top, xres, yres)
            
            tiff_file = os.path.splitext(os.path.join(subdir, file))[0] + '_planting.tif'
            with rasterio.open(tiff_file, 'w', driver='GTiff', height=ny, width=nx,
                   count=1, dtype=var_planting.dtype, crs=crs, transform=transform, compress=compression) as dst:
                       dst.write(var_planting.values, 1)
                       
            tiff_file = os.path.splitext(os.path.join(subdir, file))[0] + '_maturity.tif'
            with rasterio.open(tiff_file, 'w', driver='GTiff', height=ny, width=nx,
                   count=1, dtype=var_maturity.dtype, crs=crs, transform=transform, compress=compression) as dst:
                       dst.write(var_maturity.values, 1)
            
            input_file = tiff_file
            output_file = os.path.join(Resam_fold, file)
            gdal.Warp(output_file, input_file, xRes=xres, yRes=yres)

Find the average value of crop calander (planting and maturity day) for irrigated and rainfed crops per each polygon boundary and save it as a dataframe (Using zonal statistics)

In [7]:

import rasterstats

def extract_zonal_stats(raster_path, shapefile_path, unit_code_field):
    # Extract zonal statistics using rasterstats library
    stats = rasterstats.zonal_stats(shapefile_path, raster_path, stats="mean", nodata=-9999, geojson_out=True)

    # Create a dictionary to store the mean values
    mean_values_dict = {}

    # Loop over each feature in the geojson
    for feature in stats:
        # Extract the unit_code value
        unit_code = str(feature['properties'][unit_code_field])

        # Extract the mean value from the feature properties
        mean_value = feature['properties']['mean']

        # Store the mean value in the dictionary
        mean_values_dict[unit_code] = mean_value

    return mean_values_dict

# Define the path to the directory containing the subfolders
root_dir = r"D:\\PhD_Udel\\MICRA_2015\\Input Data\\Crop_Calander"

# Loop over each subfolder in the root directory
for subfolder in os.listdir(root_dir):
    subfolder_path = os.path.join(root_dir, subfolder)

    # Create an empty dictionary to store the mean values for each subfolder
    mean_values_dict = {}

    for file in os.listdir(subfolder_path):
        # Check if the file is a TIFF file
        if file.endswith('.tif'):
            tiff_path = os.path.join(subfolder_path, file)
            col_name = os.path.splitext(os.path.basename(tiff_path))[0]

            # Check if a corresponding SHP file exists
        if file.endswith('.shp'):
            shp_path = os.path.join(subfolder_path, file)
            if not os.path.exists(shp_path):
                continue

        # Extract the mean values for the current TIFF and SHP files
        current_mean_values = extract_zonal_stats(tiff_path, shp_path, 'unit_code')

        # Add the current mean values to the overall mean values dictionary
        mean_values_dict.update(current_mean_values)

        # Create a Pandas DataFrame from the mean values
        df = pd.DataFrame.from_dict(mean_values_dict, orient='index', columns=[col_name])
        df.index = df.index.str.replace("_", "")
        filename = os.path.join("D:\\PhD_Udel\\MICRA_2015\\Input Data\\Crop_Calander\\CCC", f"{col_name}.csv")
        df.to_csv(filename, index_label='unit_code')

        # Clear the mean_values_dict for the next subfolder
        mean_values_dict = {}

In [10]:
from datetime import datetime
root_directory = "D:\\PhD_Udel\\MICRA_2015\\Input Data\\Crop_Calander\CCC"

# Initialize an empty dataframe to store the merged data
merged_df = pd.DataFrame()

# Define the function to convert day-of-the-year values to month values
def convert_day_to_month(day):
    if day > 120 and day <= 151:
        return 5
    elif day > 0 and day <= 31:
        return 1
    elif day > 31 and day <= 59:
        return 2
    elif day > 59 and day <= 90:
        return 3
    elif day > 90 and day <= 120:
        return 4
    elif day > 151 and day <= 181:
        return 6
    elif day > 181 and day <= 212:
        return 7
    elif day > 212 and day <= 243:
        return 8
    elif day > 243 and day <= 273:
        return 9
    elif day > 273 and day <= 304:
        return 10
    elif day > 304 and day <= 334:
        return 11
    elif day > 334 and day <= 365:
        return 12
    else:
        return -1

# Loop through all files in the root directory
for filename in os.listdir(root_directory):
    filepath = os.path.join(root_directory, filename)
    
    # Check if the file is a CSV and contains data
    if filename.endswith(".csv") and os.path.getsize(filepath) > 0:
        # Read the CSV file into a dataframe
        df = pd.read_csv(filepath)
# print(df)
        
        # Convert the day-of-the-year values to month values using the function
        for col in df.columns:
            if col != "unit_code":
                df[col] = df[col].apply(convert_day_to_month)

        # If the merged dataframe is empty, set it to the current dataframe
        if merged_df.empty:
            merged_df = df
#             print(merged_df)
        else:
            # Merge the current dataframe with the merged dataframe based on the "unit_code" column
            merged_df = pd.merge(merged_df, df, on="unit_code")
#         print(merged_df)

# Reset the index of the merged dataframe
merged_df = merged_df.reset_index(drop=True)
merged_df.to_csv("D:\\PhD_Udel\\MICRA_2015\\Input Data\\Crop_Calander\CCC\\merged_df.csv")
merged_df.head(30)
#Load the dataframe with a target district crop specific growiing area and crop calander
# df_area= pd.read_excel("D:\\PhD_Udel\\MICRA_2015\\Asia\\India\\Growing area India\\Growing_Area\\2005_Ir.xlsx")
# df_area.index = df_area.index.astype(int)
# df_merged = pd.merge(df_area, merged_df, left_index=True, right_on='unit_code')
# df_merged.head(30)

,unit_code,bar_ir_maturity,bar_ir_planting,bea_ir_maturity,bea_ir_planting,cas_ir_maturity,cas_ir_planting,cot_ir_planting,cot_ir__maturity,mai_ir_maturity,...,sor_ir_maturity,sor_ir_planting,soy_ir_maturity,soy_ir_planting,sun_ir_maturity,sun_ir_planting,swh_ir_maturity,swh_ir_planting,wwh_ir_maturity,wwh_ir_planting
0,3560001,11,7,11,7,3,11,6,1,9,...,11,7,10,7,11,7,3,10,3,11
1,3560002,11,7,2,10,3,11,6,12,9,...,11,7,10,7,11,7,3,10,3,11
2,3560003,11,7,2,10,3,11,6,12,9,...,11,7,10,7,11,7,3,10,3,11
3,3560004,11,7,3,9,3,11,6,1,9,...,4,12,10,7,11,7,2,10,3,11
4,3560005,11,7,11,8,3,11,6,1,9,...,4,12,10,7,11,7,2,10,3,11
5,3560006,11,7,2,10,3,11,6,1,9,...,4,12,10,7,11,7,3,10,3,11
6,3560007,11,7,2,10,3,11,6,1,9,...,4,12,10,7,11,7,3,10,3,11
7,3560008,11,7,2,10,3,11,6,1,9,...,4,12,10,7,11,7,3,10,3,11
8,3560009,11,7,2,10,3,11,6,1,9,...,4,12,10,7,11,7,2,10,3,11
9,3560010,11,7,4,9,3,11,6,1,9,...,4,12,10,7,11,7,2,10,3,11


In [3]:
# Merge the two dataframes on 'unit_code'
df_area= pd.read_csv("D:\\PhD_Udel\\MICRA_2015\\DOwnscaling\\India\\Condensed Crop Calander\\CCC_final\\SPAM\\New\\2005_ir_merg.csv")
df_dates=pd.read_csv("D:\\PhD_Udel\\MICRA_2015\\Input Data\\Crop_Calander\\CCC\\merged_df.csv")
# Merge the dataframes on 'unit_code'
merged_df = pd.merge(df_area, df_dates, on='unit_code')
# Create a dictionary to map column names to planting/maturity dates
date_dict = {'Barley': ('bar_ir_planting', 'bar_ir_maturity'),
             'Beans': ('bar_ir_planting', 'bar_ir_maturity'),
             'Cassava': ('cas_ir_planting', 'cas_ir_maturity'),
             'Cotton': ('cot_ir_planting', 'cot_ir__maturity'),
             'Groundnuts': ('nut_ir_planting', 'nut_ir_maturity'),
             'Maize': ('mai_ir_planting', 'mai_ir_maturity'),
             'Millet': ('mil_ir_planting', 'mil_ir_maturity'),
             'Potatoes': ('pot_ir_planting', 'pot_ir_maturity'),
             'Peas': ('pea_ir_planting', 'pea_ir_maturity'),
             'Rapeseed': ('rap_ir_planting', 'rap_ir_maturity'),
             'Rice1': ('ri1_ir_planting', 'ri1_ir_maturity'),
             'Rice2': ('ri2_ir_planting', 'ri2_ir_maturity'),
             'Sugarcane': ('sgc_ir_planting', 'sgc_ir_maturity'),
             'Sorghum': ('sor_ir_planting', 'sor_ir_maturity'),
             'Soybeans': ('soy_ir_planting', 'soy_ir_maturity'),
             'Sunflower': ('sun_ir_planting', 'sun_ir_maturity'),
             'Wheat': ('wwh_ir_planting', 'wwh_ir_maturity')}
# Iterate over rows in merged dataframe and update planting/maturity dates in df_area
for index, row in merged_df.iterrows():
    crop = row['Crop']
    unit_code = row['unit_code']
    planting_date_col, maturity_date_col = date_dict.get(crop, (None, None))
    if planting_date_col and maturity_date_col:
        planting_month = row[planting_date_col]
        maturity_month = row[maturity_date_col]
        df_area.loc[(df_area['unit_code'] == unit_code) & (df_area['Crop'] == crop), 'Planting_Month'] = planting_month
        df_area.loc[(df_area['unit_code'] == unit_code) & (df_area['Crop'] == crop), 'Maturity_Month'] = maturity_month
df_area.to_csv("D:\\PhD_Udel\\MICRA_2015\\DOwnscaling\\India\\Condensed Crop Calander\\CCC_final\\SPAM\\New\\2005_ir_merg_1.csv")

Merge the growing area dataframe with the crop calander dataframe

In [2]:
root_folder = r"D:\\PhD_Udel\\MICRA_2015\\Input Data\CC_Rainfed"
crs = pyproj.CRS.from_epsg(4326)
Resam_fold = r"D:\\PhD_Udel\\MICRA_2015\\Input Data\\CC_Rainfed\Resmaple"
xres, yres = 0.00833, 0.00833
compression = "LZW"

for subdir, dirs, files in os.walk(root_folder):
    for file in files:
        if file.endswith('.nc4'):
            nc_file = xr.open_dataset(os.path.join(subdir, file))
            var_planting = nc_file.variables["planting_day"]
            var_maturity = nc_file.variables["maturity_day"]
            
            x = nc_file.coords['lon'].values
            y = nc_file.coords['lat'].values
            left, bottom = x.min(), y.min()
            right, top = x.max(), y.max()
            nx = int((right - left) / xres)
            ny = int((top - bottom) / yres)
            
            transform = rt.from_origin(left, top, xres, yres)
            
            tiff_file = os.path.splitext(os.path.join(subdir, file))[0] + '_planting.tif'
            with rasterio.open(tiff_file, 'w', driver='GTiff', height=ny, width=nx,
                   count=1, dtype=var_planting.dtype, crs=crs, transform=transform, compress=compression) as dst:
                       dst.write(var_planting.values, 1)
                       
            tiff_file = os.path.splitext(os.path.join(subdir, file))[0] + '_maturity.tif'
            with rasterio.open(tiff_file, 'w', driver='GTiff', height=ny, width=nx,
                   count=1, dtype=var_maturity.dtype, crs=crs, transform=transform, compress=compression) as dst:
                       dst.write(var_maturity.values, 1)
            
            input_file = tiff_file
            output_file = os.path.join(Resam_fold, file)
            gdal.Warp(output_file, input_file, xRes=xres, yRes=yres)

In [6]:

import rasterstats

def extract_zonal_stats(raster_path, shapefile_path, unit_code_field):
    # Extract zonal statistics using rasterstats library
    stats = rasterstats.zonal_stats(shapefile_path, raster_path, stats="mean", nodata=-9999, geojson_out=True)

    # Create a dictionary to store the mean values
    mean_values_dict = {}

    # Loop over each feature in the geojson
    for feature in stats:
        # Extract the unit_code value
        unit_code = str(feature['properties'][unit_code_field])

        # Extract the mean value from the feature properties
        mean_value = feature['properties']['mean']

        # Store the mean value in the dictionary
        mean_values_dict[unit_code] = mean_value

    return mean_values_dict

# Define the path to the directory containing the subfolders
root_dir = r"D:\\PhD_Udel\\MICRA_2015\\Input Data\CC_Rainfed"

# Loop over each subfolder in the root directory
for subfolder in os.listdir(root_dir):
    subfolder_path = os.path.join(root_dir, subfolder)

    # Create an empty dictionary to store the mean values for each subfolder
    mean_values_dict = {}

    for file in os.listdir(subfolder_path):
        # Check if the file is a TIFF file
        if file.endswith('.tif'):
            tiff_path = os.path.join(subfolder_path, file)
            col_name = os.path.splitext(os.path.basename(tiff_path))[0]

            # Check if a corresponding SHP file exists
        if file.endswith('.shp'):
            shp_path = os.path.join(subfolder_path, file)
            if not os.path.exists(shp_path):
                continue

        # Extract the mean values for the current TIFF and SHP files
        current_mean_values = extract_zonal_stats(tiff_path, shp_path, 'unit_code')

        # Add the current mean values to the overall mean values dictionary
        mean_values_dict.update(current_mean_values)

        # Create a Pandas DataFrame from the mean values
        df = pd.DataFrame.from_dict(mean_values_dict, orient='index', columns=[col_name])
        df.index = df.index.str.replace("_", "")
        filename = os.path.join("D:\\PhD_Udel\\MICRA_2015\\Input Data\\CC_Rainfed\\CCC", f"{col_name}.csv")
        df.to_csv(filename, index_label='unit_code')

        # Clear the mean_values_dict for the next subfolder
        mean_values_dict = {}

In [72]:
from datetime import datetime
root_directory = "D:\\PhD_Udel\\MICRA_2015\\Input Data\\CC_Rainfed\\CCC"

# Initialize an empty dataframe to store the merged data
merged_df = pd.DataFrame()

# Define the function to convert day-of-the-year values to month values
def convert_day_to_month(day):
    if day > 0 and day <= 31:
        return 1
    elif day > 31 and day <= 59:
        return 2
    elif day > 59 and day <= 90:
        return 3
    elif day > 90 and day <= 120:
        return 4
    elif day > 120 and day <= 151:
        return 5
    elif day > 151 and day <= 181:
        return 6
    elif day > 181 and day <= 212:
        return 7
    elif day > 212 and day <= 243:
        return 8
    elif day > 243 and day <= 273:
        return 9
    elif day > 273 and day <= 304:
        return 10
    elif day > 304 and day <= 334:
        return 11
    elif day > 334 and day <= 365:
        return 12
    else:
        return -1

# Loop through all files in the root directory
for filename in os.listdir(root_directory):
    filepath = os.path.join(root_directory, filename)
    
    # Check if the file is a CSV and contains data
    if filename.endswith(".csv") and os.path.getsize(filepath) > 0:
        # Read the CSV file into a dataframe
        df = pd.read_csv(filepath)
        
        # Convert the day-of-the-year values to month values using the function
        for col in df.columns:
            if col != "unit_code":
                df[col] = df[col].apply(convert_day_to_month)

        # If the merged dataframe is empty, set it to the current dataframe
        if merged_df.empty:
            merged_df = df
        else:
            # Merge the current dataframe with the merged dataframe based on the "unit_code" column
            merged_df = pd.merge(merged_df, df, on="unit_code")

# Reset the index of the merged dataframe
merged_df = merged_df.reset_index(drop=True)
# os.chdir("D:\\PhD_Udel\\MICRA_2015\\Input Data\\CC_Rainfed\\CCC")
# merged_df.to_csv("merged_df.csv")
# merged_df.head(30)


,unit_code,bar_rf_g_planting,bar_rf_maturity,bea_rf_maturity,bea_rf_planting,cas_rf_maturity,cas_rf_planting,cot_rf_maturity,cot_rf_planting,mai_rf_planting,...,sor_rf_maturity,sor_rf_planting,soy_rf_maturity,soy_rf_planting,sun_rf_maturity,sun_rf__planting,swh_rf_maturity,swh_rf_planting,wwh_rf_maturity,wwh_rf_planting
0,3560001,7,11,2,10,10,3,1,6,6,...,10,7,10,7,11,7,3,10,3,11
1,3560002,7,11,2,10,10,3,12,6,6,...,10,7,11,7,11,7,3,10,3,11
2,3560003,7,11,2,10,10,3,12,6,6,...,10,7,10,7,11,7,3,10,3,11
3,3560004,7,11,3,9,10,3,1,6,6,...,10,5,10,7,11,7,2,10,3,11
4,3560005,7,11,11,8,10,3,1,6,6,...,10,5,11,7,11,7,2,10,3,11
5,3560006,7,11,2,10,10,3,1,6,6,...,11,5,10,7,11,7,3,10,3,11
6,3560007,7,11,2,10,10,3,1,6,6,...,10,5,10,7,11,7,3,10,3,11
7,3560008,7,11,2,10,10,3,1,6,6,...,10,5,10,7,11,7,3,10,3,11
8,3560009,7,11,2,10,10,3,1,6,6,...,10,5,10,7,11,7,2,10,3,11
9,3560010,7,11,4,9,10,3,1,6,6,...,10,5,10,7,11,7,2,10,3,11


conver the percentage value of hyde 2005 cropland extent to absolute value

In [ ]:
#conver the percentage value of hyde 2005 cropland extent to absolute value
# Convert percentage values to absolute values for each input raster
# Define input and output folder paths
input_folder = r"D:\\PhD_Udel\\MICRA_2015\\Input Data\\AH Grid\\HYDE\\LU\\Cropland"
output_folder = r"D:\\PhD_Udel\\MICRA_2015\\Input Data\\AH Grid\\HYDE\\LU\\Cropland\\Absolute"

# Convert percentage values to absolute values for each input raster
for file in os.listdir(input_folder):
    if file.endswith(".tif"):
        # Open input raster and read data
        input_file = os.path.join(input_folder, file)
        with rasterio.open(input_file) as src:
            data = src.read(1)

            # Convert percentage values to absolute values
            abs_data = data * pixel_areas 

            # Create output raster file
            output_file = os.path.join(output_folder, file.replace(".tif", "_absolute.tif"))
            profile = src.profile.copy()
            profile.update(dtype=rasterio.float64, count=1)

            # Write output raster file
            with rasterio.open(output_file, "w", **profile) as dst:
                dst.write(abs_data.astype(rasterio.float64), 1)

preparing area of 5-arcminute grid cell

In [ ]:
import numpy as np
import rasterio
from rasterio.transform import from_origin
import rasterio.crs

# Open the raster file
fp = r"D:\\PhD_Udel\\MICRA_2015\\DOwnscaling\\India\\2005\\CLE\\CLE_05.tif"
with rasterio.open(fp) as testif:
    rast = testif.read(1)
    gt = testif.transform
    pix_width = gt[0]
    ulX = gt[2]
    ulY = gt[5]
    rows = testif.height
    cols = testif.width
    lrX = ulX + gt[0] * cols
    lrY = ulY + gt[4] * rows

# Generate latitudes array
lats = np.linspace(ulY, lrY, rows+1)

# WGS84 ellipsoid constants
a = 6378137
b = 6356752.3142

# Degrees to radians
lats = lats * np.pi/180

# Intermediate vars
e = np.sqrt(1-(b/a)**2)
sinlats = np.sin(lats)
zm = 1 - e * sinlats
zp = 1 + e * sinlats

# Distance between meridians
q = pix_width / 360

# Compute areas for each latitude in square km
areas_to_equator = np.pi * b**2 * ((2 * np.arctanh(e * sinlats) / (2 * e) + sinlats / (zp * zm))) / 10**6
areas_between_lats = np.diff(areas_to_equator)
areas_cells = np.abs(areas_between_lats) * q

# Create the area grid
areagrid = np.transpose(np.matlib.repmat(areas_cells, cols, 1))
pixel_areas = areagrid * 100  # Convert Sqkm to hectares

# Define the transform for the output raster
transform = from_origin(ulX, ulY, pix_width, gt[4])

# Define the CRS for the output raster
crs = rasterio.crs.CRS.from_epsg(4326)

# Output file path
output_fp = r"D:\\PhD_Udel\\MICRA_2015\\DOwnscaling\\India\\2005\\Area\\Grid_area.tif"

# Save the area grid as a new raster
with rasterio.open(
    output_fp,
    'w',
    driver='GTiff',
    height=pixel_areas.shape[0],
    width=pixel_areas.shape[1],
    count=1,
    dtype=pixel_areas.dtype,
    crs=crs,
    transform=transform,
) as dst:
    dst.write(pixel_areas, 1)

print(np.max(pixel_areas), np.min(pixel_areas))
